크롤링

In [1]:
import selenium #셀레니움
import pandas as pd #csv를 읽고 dataframe을 사용하기 위한 pandas
from selenium import webdriver #브라우저를 띄우고 컨트롤하기 위한 webdriver
from selenium.webdriver.common.keys import Keys #브라우저에 키입력 용
from selenium.webdriver.common.by import By #webdriver를 이용해 태그를 찾기 위함
from selenium.webdriver.support.ui import WebDriverWait #Explicitly wait을 위함
from webdriver_manager.chrome import ChromeDriverManager #크롬에서 크롤링 진행 크롬 웹 드라이버 설치시 필요
from selenium.webdriver.support import expected_conditions as EC #브라우저에 특정 요소 상태 확인을 위해
from bs4 import BeautifulSoup #브라우저 태그를 가져오고 파싱하기 위함
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException #예외처리를 위한 예외들
from time import sleep
#크롬 드라이버 최신 버전 https://googlechromelabs.github.io/chrome-for-testing/

In [23]:
## 이 코드는 전에 네이버 API로 한거

df=pd.read_csv("C:/Users/taejin/Desktop/2024-1-DSCD-ASAP-4/result/multi_result_서울특별시_중구_필동_카페_1.csv",encoding="CP949")
df.drop('Unnamed: 0',axis=1,inplace=True)
#df=df.head(50)
df_no_duplicates = df.drop_duplicates(subset=df.columns)
#주소 간소화시켜주는 함수(ex) 명달로 9길)
def get_addrs(x):
    x1 = x.split(' ')
    return " ".join(x1[2:4])

df_no_duplicates['주소2'] = df['주소'].apply(get_addrs)

import pandas as pd
import requests

#위경도 변환 함수
def get_lat_lng(address):
    client_id = 's8pego1ogd' #각자 네이버 api 받은것
    client_secret = 'IDwfqb34hdFls7BxU0V8WknUzJ0HrQn46NUjfBn9'
    
    url = f'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query={address}'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret
    }
    
    response = requests.get(url, headers=headers)
    data = response.json()
    
    if 'addresses' in data:
        if len(data['addresses']) > 0:
            latitude = data['addresses'][0]['y']
            longitude = data['addresses'][0]['x']
            return latitude, longitude
        else:
            return None, None
    else:
        return None, None

def convert_address_to_lat_lng(df, address_column):
    latitudes = []
    longitudes = []
    for index, row in df.iterrows():
        address = row[address_column]
        if address:
            latitude, longitude = get_lat_lng(address)
            latitudes.append(latitude)
            longitudes.append(longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)
    df['Latitude'] = latitudes
    df['Longitude'] = longitudes


# 주소를 위도와 경도로 변환하여 데이터프레임에 추가
convert_address_to_lat_lng(df_no_duplicates, '주소2')

df_no_duplicates['Latitude'] = df_no_duplicates['Latitude'].astype('float')
df_no_duplicates['Longitude'] = df_no_duplicates['Longitude'].astype('float')

In [24]:
df_no_duplicates

,가게명,업종,전화번호,주소,별점,방문자/블로그 리뷰,프랜차이즈여부,주소2,Latitude,Longitude
0,커피인쇄소,카페,070-2345-6789,서울 중구 퇴계로27길 12-5 2층,별점 4.63,방문자리뷰 28 블로그리뷰 80,X,퇴계로27길 12-5,37.561794,126.992031
1,더스크커피,"카페,디저트",0507-0289-5027,서울 중구 퇴계로32길 14 더스크커피,-,방문자리뷰 293 블로그리뷰 122,X,퇴계로32길 14,37.560390,126.992495
2,스타벅스 충무로점,카페,1522-3232,서울 중구 충무로 3,-,방문자리뷰 436 블로그리뷰 30,X,충무로 3,37.561491,126.992878
3,리바운드 충무로점,스터디카페,0507-1472-7411,"서울 중구 충무로 3 112호, B129호~",-,방문자리뷰 3 블로그리뷰 24,X,충무로 3,37.561491,126.992878
4,팀앤메리,"카페,디저트",02-2269-5430,서울 중구 퇴계로27길 12-3,별점 4.57,방문자리뷰 252 블로그리뷰 46,X,퇴계로27길 12-3,37.561807,126.991810
...,...,...,...,...,...,...,...,...,...,...
398,몬떼비서울 충무로점,"카페,디저트",0507-1378-8128,서울 중구 수표로2길 15 1층,-,방문자리뷰 693 블로그리뷰 43,X,수표로2길 15,37.562229,126.991346
399,카페아이엠티 을지로시그니처타워점,카페,0507-1337-0892,서울 중구 청계천로 100 1층,-,방문자리뷰 311 블로그리뷰 29,X,청계천로 100,37.567651,126.988445
400,스타벅스 종로관철점,카페,1522-3232,서울 종로구 종로12길 21,별점 4.51,"방문자리뷰 3,531 블로그리뷰 112",X,종로12길 21,37.569085,126.986007
401,젤라또로플 두타본점,"카페,디저트",0507-1343-5128,서울 중구 장충단로 275 두산타워 B2층 FB115호(을지로6가),-,방문자리뷰 288 블로그리뷰 5,X,장충단로 275,37.568891,127.008783


In [33]:
def extract_and_sum(review_str):
    import re
    numbers = re.findall(r'\d+', str(review_str))  # 문자열에서 숫자만 추출
    numbers = map(int, numbers)  # 추출한 숫자를 정수형으로 변환
    return sum(numbers)  # 숫자 합산

def extract_rating(rating_str):
    import re
    number = re.findall(r'\d+\.\d+', rating_str)  # 문자열에서 소수점을 포함한 숫자 추출
    if number:
        return float(number[0])  # 추출한 숫자를 실수형으로 변환
    return None  # 숫자가 없을 경우 None 반환

# 새로운 열 '리뷰 합계' 추가
df_no_duplicates['리뷰 합계'] = df_no_duplicates['방문자/블로그 리뷰'].apply(extract_and_sum)

#별점 뽑기
df_no_duplicates['별점_1'] = df_no_duplicates['별점'].apply(extract_rating)

df_no_duplicates

,가게명,업종,전화번호,주소,별점,방문자/블로그 리뷰,프랜차이즈여부,주소2,Latitude,Longitude,리뷰 합계,별점_1
0,커피인쇄소,카페,070-2345-6789,서울 중구 퇴계로27길 12-5 2층,별점 4.63,방문자리뷰 28 블로그리뷰 80,X,퇴계로27길 12-5,37.561794,126.992031,108,4.63
1,더스크커피,"카페,디저트",0507-0289-5027,서울 중구 퇴계로32길 14 더스크커피,-,방문자리뷰 293 블로그리뷰 122,X,퇴계로32길 14,37.560390,126.992495,415,NaN
2,스타벅스 충무로점,카페,1522-3232,서울 중구 충무로 3,-,방문자리뷰 436 블로그리뷰 30,X,충무로 3,37.561491,126.992878,466,NaN
3,리바운드 충무로점,스터디카페,0507-1472-7411,"서울 중구 충무로 3 112호, B129호~",-,방문자리뷰 3 블로그리뷰 24,X,충무로 3,37.561491,126.992878,27,NaN
4,팀앤메리,"카페,디저트",02-2269-5430,서울 중구 퇴계로27길 12-3,별점 4.57,방문자리뷰 252 블로그리뷰 46,X,퇴계로27길 12-3,37.561807,126.991810,298,4.57
...,...,...,...,...,...,...,...,...,...,...,...,...
398,몬떼비서울 충무로점,"카페,디저트",0507-1378-8128,서울 중구 수표로2길 15 1층,-,방문자리뷰 693 블로그리뷰 43,X,수표로2길 15,37.562229,126.991346,736,NaN
399,카페아이엠티 을지로시그니처타워점,카페,0507-1337-0892,서울 중구 청계천로 100 1층,-,방문자리뷰 311 블로그리뷰 29,X,청계천로 100,37.567651,126.988445,340,NaN
400,스타벅스 종로관철점,카페,1522-3232,서울 종로구 종로12길 21,별점 4.51,"방문자리뷰 3,531 블로그리뷰 112",X,종로12길 21,37.569085,126.986007,646,4.51
401,젤라또로플 두타본점,"카페,디저트",0507-1343-5128,서울 중구 장충단로 275 두산타워 B2층 FB115호(을지로6가),-,방문자리뷰 288 블로그리뷰 5,X,장충단로 275,37.568891,127.008783,293,NaN


In [35]:
from folium.plugins import MarkerCluster
from folium import Marker
import folium

lat_m = df_no_duplicates['Latitude'].mean()
long_m = df_no_duplicates['Longitude'].mean()

m = folium.Map(location=[lat_m, long_m], zoom_start=12)

mc = MarkerCluster()

for _, row in df_no_duplicates.iterrows():
    popup_content = f'''
    
    <div style="white-space: nowrap; max-width: 250px;">
    <b>🏠:</b> {row['가게명']}<br>
    <b>✔:</b> {row['업종']}<br>
    <b>📍:</b> {row['주소']}<br>
    <b>⭐:</b> {row['별점_1']}<br>
    <b>👀:</b> {row['리뷰 합계']}
    </div>
    '''
    mc.add_child(
        Marker(location=[row['Latitude'], row['Longitude']],
               popup=popup_content
              )
    ).add_to(m)

m


In [37]:
m.save('서울특별시_중구_필동_카페.html')